In [483]:
# Go through and audit Model

# Talk about my classwork and what I will eventually be doing in module 3

# What are next steps for a project that doesn't have historical data on-hand.

# How to get appropriate historical data from each data provider? Set expectations upfront

# Model to determine which is better- RRP or pubsub

# Standardization of data requirements - getting that upfront from our data providers is imperitive

# How to build 'historical' data sets by pulling current data

# Have the contract acknowledge that if volatility exceeds x range then possibly readjust.

# Is there an opportunity to make make monitoring tools to keep tabs on that.

# ETH/ Docker container to figure this out?

In [499]:
import requests
import json
import pandas as pd
import os
import numpy as np
import hvplot.pandas
from utils.utils import coin_gecko_cleaner
from datetime import date
from datetime import datetime
%matplotlib inline

In [500]:
BAYC_df = pd.read_csv('BAYCPriceHistory.csv')
display(BAYC_df)

,\tdatetime\tprice,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,0\t2021-07-30T00:00:00.000Z\t5.5853,NaN,NaN,NaN,NaN
1,1\t2021-07-30T08:00:00.000Z\t5.4953,NaN,NaN,NaN,NaN
2,2\t2021-07-30T16:00:00.000Z\t5.4547,NaN,NaN,NaN,NaN
3,3\t2021-07-31T00:00:00.000Z\t5.5301,NaN,NaN,NaN,NaN
4,4\t2021-07-31T08:00:00.000Z\t6.3304,NaN,NaN,NaN,NaN
...,...,...,...,...,...
946,946\t2022-06-10T08:00:00.000Z\t91.5888,NaN,NaN,NaN,NaN
947,947\t2022-06-10T16:00:00.000Z\t90.2698,NaN,NaN,NaN,NaN
948,948\t2022-06-11T00:00:00.000Z\t89.2813,NaN,NaN,NaN,NaN
949,949\t2022-06-11T08:00:00.000Z\t88.0673,NaN,NaN,NaN,NaN


In [501]:
BAYC_df['\tdatetime\tprice'].dtypes

dtype('O')

In [502]:
BAYC_df['new'] = BAYC_df['\tdatetime\tprice'].str.split('\t')
display(BAYC_df)

,\tdatetime\tprice,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,new
0,0\t2021-07-30T00:00:00.000Z\t5.5853,NaN,NaN,NaN,NaN,"[0, 2021-07-30T00:00:00.000Z, 5.5853]"
1,1\t2021-07-30T08:00:00.000Z\t5.4953,NaN,NaN,NaN,NaN,"[1, 2021-07-30T08:00:00.000Z, 5.4953]"
2,2\t2021-07-30T16:00:00.000Z\t5.4547,NaN,NaN,NaN,NaN,"[2, 2021-07-30T16:00:00.000Z, 5.4547]"
3,3\t2021-07-31T00:00:00.000Z\t5.5301,NaN,NaN,NaN,NaN,"[3, 2021-07-31T00:00:00.000Z, 5.5301]"
4,4\t2021-07-31T08:00:00.000Z\t6.3304,NaN,NaN,NaN,NaN,"[4, 2021-07-31T08:00:00.000Z, 6.3304]"
...,...,...,...,...,...,...
946,946\t2022-06-10T08:00:00.000Z\t91.5888,NaN,NaN,NaN,NaN,"[946, 2022-06-10T08:00:00.000Z, 91.5888]"
947,947\t2022-06-10T16:00:00.000Z\t90.2698,NaN,NaN,NaN,NaN,"[947, 2022-06-10T16:00:00.000Z, 90.2698]"
948,948\t2022-06-11T00:00:00.000Z\t89.2813,NaN,NaN,NaN,NaN,"[948, 2022-06-11T00:00:00.000Z, 89.2813]"
949,949\t2022-06-11T08:00:00.000Z\t88.0673,NaN,NaN,NaN,NaN,"[949, 2022-06-11T08:00:00.000Z, 88.0673]"


In [503]:
def extract_first(item):
    return item[0]

def extract_second (item):
    return item[1]

def extract_third(item):
    return item[2]

BAYC_df = BAYC_df.drop(columns=['\tdatetime\tprice','Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']) 
BAYC_df['col_1'] = BAYC_df['new'].apply(extract_first)
BAYC_df['Date/Time'] = BAYC_df['new'].apply(extract_second)
BAYC_df['Price'] = BAYC_df['new'].apply(extract_third)
BAYC_df = BAYC_df.drop(columns=['new'])
BAYC_df = BAYC_df.drop(columns=['col_1'])
BAYC_df['Date/Time'] = BAYC_df['Date/Time'].str.replace('T',', ')
BAYC_df['Date/Time'] = BAYC_df['Date/Time'].str.replace('Z','')
BAYC_df['Price'] = pd.to_numeric(BAYC_df['Price'])
BAYC_df['Date/Time'] = pd.to_datetime(BAYC_df['Date/Time'])
BAYC_df

,Date/Time,Price
0,2021-07-30 00:00:00.000,5.5853
1,2021-07-30 08:00:00.000,5.4953
2,2021-07-30 16:00:00.000,5.4547
3,2021-07-31 00:00:00.000,5.5301
4,2021-07-31 08:00:00.000,6.3304
...,...,...
946,2022-06-10 08:00:00.000,91.5888
947,2022-06-10 16:00:00.000,90.2698
948,2022-06-11 00:00:00.000,89.2813
949,2022-06-11 08:00:00.000,88.0673


In [504]:
## Define Desired Deviation

deviation_high = 1.04
deviation_low = .96


In [505]:
## List Initiation
current_price = BAYC_df['Price'].iloc[0]
print(current_price)


5.5853


In [506]:
# Define Time Period
BAYC_five_months = BAYC_df[BAYC_df['Date/Time'].between('2022-01-01','2022-06-01')]

display(BAYC_five_months)

,Date/Time,Price
465,2022-01-01 00:00:00,60.0000
466,2022-01-01 08:00:00,63.6350
467,2022-01-01 16:00:00,66.1667
468,2022-01-02 00:00:00,67.6269
469,2022-01-02 08:00:00,67.7529
...,...,...
914,2022-05-30 16:00:00,86.0623
915,2022-05-31 00:00:00,85.7541
916,2022-05-31 08:00:00,85.4191
917,2022-05-31 16:00:00,85.0540


In [507]:
trigger = 0 


for price_to_check in BAYC_five_months['Price']:
    if price_to_check < (current_price * deviation_low):
        current_price=price_to_check
        trigger= trigger+1
        
        
    if price_to_check > (current_price * deviation_high):
        current_price=price_to_check
        trigger=trigger+1
        

print(trigger)

79


In [510]:
monthly_average = trigger/5
print(monthly_average)

15.8
